In [1]:
import os
import numpy as np
from nilearn.image import smooth_img
from nilearn import plotting
from glob import glob
import sys
sys.path.append('../defacing')
from helpers.utils import load_vol
%matplotlib inline
import matplotlib.pyplot as plt

# Documentation

This document explains the various experiments and datasets contained within this folder for the project `mri-face-detector`.

```
conformed: 
contains the pre-proccessed and conformed datasets for the following:
 
 - deface: The original defaced datasets from OpenNeuro
 - face: The original faced datasets from OpenNeuro
 - face_defaced: The original faced datasets defaced using pydeface from OpenNeuro

experiment_data_all: 
contains the models, tfrecords, data-split csv's and results trained on the entire datasets (faced, defaced and faced_defaced volumes)

experiment_faced_defaced:
contains the models, tfrecords, data-split csv's and results trained on a subset of datasets (only the faced and faced_defaced volumes)

experiment_faced_refaced: 
contains the models, tfrecords, data-split csv's and results trained on a subset of datasets (only 100 faced, faced_defaced and faced_refaced volumes). This experiment was conducted to understand the possibility/repurcussions of using the refacing tools.

masks:

preprocessing:
contains only the pre-processed volume data. The pre-processing include min-max normalization, median standardization and clipping values to the 90th percentile.
  
  - deface, face, face_defaced: explained under section conformed.
  - save_plots: Visual plots generated to guage the validity of the pre-processing. Each volume figure was generated using nilearn.

refaced:
  - afni-latest.simg: the singularity image to run afni (Precompiled binary linux_openmp_64: Jul 31 2020 (Version AFNI_20.2.08 'Aulus Vitellius')
  - orig_defaced: the refaced volumes generated using afni-reface-run for the original defaced datasets from OpenNeuro
  - orig_faced: the refaced volumes generated using afni-reface-run for the original faced datasets that were defaced using pydeface from OpenNeuro

test_conformed:

```

In [3]:
ROOT_PATH = '/work/06850/sbansal6/maverick2/mriqc-shared/refaced/'
# ROOT_PATH = '../'
SAVE_ROOT = os.path.join(ROOT_PATH, 'save_plots')
DISTRIBUTION = load_vol('../defacing/helpers/distribution.nii.gz')[0]
DISTRIBUTION /= DISTRIBUTION.sum()

sampler = lambda n: np.array([ np.unravel_index(
          np.random.choice(np.arange(np.prod(DISTRIBUTION.shape)),
                                      p = DISTRIBUTION.ravel()),
                                      DISTRIBUTION.shape) for _ in range(n)]) 


def plot(img_path, nslices=20, dataset=None):
    save_path = os.path.join(ROOT_PATH, dataset, 'save_plots')
    print(print ("[INFO]: Save Path {}".format(save_path)))
    os.makedirs(save_path, exist_ok=True)
    coordinates = sampler(nslices)
    img = smooth_img(img_path, fwhm=3) 
    fig, axes = plt.subplots(nrows=nslices, ncols=1, figsize=(28, int(nslices*2.5)))
        
    axes = axes.flatten()
    for i, coordinate in enumerate(coordinates):
#         print(i, coordinate)
        display = plotting.plot_anat(img, display_mode='ortho', cut_coords = coordinate, axes=axes[i])
    
    fig.savefig(os.path.join(save_path, 'vol_plot.png'))
    display.close()
    pass

def plot_all(dataset, 
             nvolumes = 25, 
             nslices = 20, 
             verbose = True):
    
    paths = glob(os.path.join(ROOT_PATH, dataset, '*.nii*'))
    paths = np.random.choice(paths, size=nvolumes) if len(paths) > nvolumes else paths
    
    for path in paths:
        print("="*10 + path + "="*10)
        plot(path, nslices = nslices, dataset = dataset)
    pass


# for ds in os.listdir(ROOT_PATH):
#     if ds == 'ds000213':
#         plot_all(ds, None)

classes = os.listdir(ROOT_PATH)
# print(classes)
exclude = ['ds002033_anat']

for dataset in classes:
    if dataset not in exclude:
        print ("[INFO]: Dataset {}".format(dataset))
        plot_all(dataset, nvolumes = 20, nslices = 10)

['ds002033_anat', 'ds002149_anat', 'ds001928_anat', 'ds002316_anat', 'ds002001_anat', 'ds002076_anat', 'ds002156_anat', 'ds002247_anat', 'ds002236_anat', 'ds001946_anat', 'ds002153_anat', 'ds001985_anat', 'ds002006_anat', 'ds002207_anat', 'ds002250_anat', 'ds001972_anat', 'ds002185_anat', 'ds002152_anat', 'afni-latest.simg', 'ds002237_anat', 'ds002080_anat', 'ds002014_anat', 'ds001980_anat', 'ds002041_anat', 'ds002011_anat', 'ds002242_anat', 'ds001923_anat', 'ds002000_anat', 'ds001978_anat', 'ds002147_anat', 'ds002013_anat', 'ds001927_anat', 'ds001926_anat', 'ds001921_anat', 'ds002016_anat', 'ds001912_anat', 'ds002320_anat']
